# Movie Budget Risk Analysis

## Business Understanding

- Stakeholders: a company wanting to enter the orginail video content creation space by opening a new movie studio.


- Task: Identify which types of films are currently the most successful and summarize these findings for stakeholders.

## Data Understanding

The data explored for this task came from two sources:

- [The Numbers](https://www.the-numbers.com/): A film data website which provided production budgeting information and worldwide gross revenue estimations for the films we analyzed. This website is operated by [OpusData](https://www.opusdata.com/home.php), and it allows access to all the financial film data via its web-based API.
    - The data from this website was stored in a csv file - 'unzippedData/tn.movie_budgets.csv' - that is linked below and included in our repository for reference.


- [IMDb](https://www.imdb.com/): A website that functions to gather and present information about the success of films based on metrics like ratings, while also contributing categorical metrics such as attributing directors to films and connecting principal actors to their work. We utilized this website to categorize films into their appropriate genres as well as to identify the directors that worked on each of the films.
    - The data from IMDb is stored as a SQL database and can also be found in our repository for refernce. We access it below using the following path: 'unzippedData/im.db'

## Data Preparation

- To begin this project, we will import all necessary python libraries that will be utilized throughout the notebook. Then we will access our data via connecting to the data files stored in our repositroy. 

In [1]:
# Importing python libraries
import pandas as pd
import sqlite3
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

#Hides warnings
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# Viewing the data sources in the repository stored in the file 'unzippedData'
! ls unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [3]:
# Reading the csv file containing The Numbers movie data and displaying the columns for an initial view of the data.
MovieBudget = pd.read_csv('unzippedData/tn.movie_budgets.csv')
MovieBudget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [4]:
# Connecting to IMDb SQL database.
conn = sqlite3.connect('unzippedData/im.db')

### Displaying the Entity Relational Diagram of all the tables and their respective columns in the IMDb database.
- This visual allows users to recognize how each of the tables are connected via designated foreign keys.
![ERD Diagram IMDB](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)

### Data Cleaning
- Through the creation of the various functions detialed below, we were able to scrub the data from our various sources. Cleaning the data allowed us to run aggregating functions in the later stages of our data analysis.

In [5]:
# Function for cleaning numbers column, removing unnecessary characters and casting the column as integers.

def clean_number_series(Series):
    Series = Series.str.replace(",", "")
    Series = Series.str.replace("$", "")
    Series = Series.astype(int)
    return Series

# Function for cleaning column headers.

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")
    
# Function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

### Applying functions to The Numbers dataset.

In [6]:
# Running the 'MovieBudget' variable through the functions.
clean_column_headers(MovieBudget)
MovieBudget['Worldwide Gross'] = clean_number_series(MovieBudget['Worldwide Gross'])
MovieBudget['Domestic Gross'] = clean_number_series(MovieBudget['Domestic Gross'])
MovieBudget['Production Budget'] = clean_number_series(MovieBudget['Production Budget'])

## Data Analysis

### Analysis Overview:
1. Identify which metric best measures the "success" of a particular film.

2. Explore the different options available to categorize films (using genre classification or decade, etc). 

3. Developing a strategy to present this data to the stakeholders. 

      - As the project unfolded, additional data cleaning and preparation occured and will be detailed below.


#### 1. Measuring Film Success
Using The Numbers dataset, we identified the columns 'Worldwide Gross' revenue and 'Production Budget' as the key metrics that we could use in our analysis. Each record in the dataset corresponds to a unique film and it includes the date of release for that film. 

Profitability Column:
  - First we created the column 'Profitability' to gain insight on exaclty how much money was earned by each film. The equation for this column subtracts the 'Production Budget' or the cost to create the movie from the revenues earned which is stored in the 'Worldwide Gross' column for each film.

In [7]:
# Adding 'Profitability' column to The Numbers dataset.
MovieBudget['Profitability'] = MovieBudget['Worldwide Gross'] - MovieBudget['Production Budget']

We determined that we could not rely on this metric alone, as this film data spans many decades and there is no indication that the financial figures provided have been adjusted for inflation.

Next we decided to engineer a ratio between the revenues earned worldwide and the cost to create the movie. 

In [8]:
# Calculating the ratio bewteen revenues earned an the cost to make the movie.
MovieBudget['ARR'] = MovieBudget['Worldwide Gross'] / MovieBudget['Production Budget']

This engineered ratio allows us to ignore additional (and potentially inaccurate) inflation adjustments over time, as each film incurred costs and revenues in it's own time, so the 'Worldwide Gross' and 'Production Budget' values may be combined and analysed without additional scaling.

In [9]:
# Checking to see if the columns were properly added.
MovieBudget

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,6.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,4.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,4.153696
...,...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,0.000000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,234495,40.082500
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662,0.267600
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,0.000000


Release Date Filter:

- Our next step was to clean the Release Date column. We determined that the month and day information was too granular for our analysis at this stage, so we isolated the release year associated with each film.

In [10]:
# Redefining the 'Release Date' column to just display the release year and then checking how many films 
#    are documented for each year.
MovieBudget_Prepared = MovieBudget.copy()
MovieBudget_Prepared['Release Date'] = MovieBudget_Prepared['Release Date'].str[-4:].astype(int)
MovieBudget_Prepared['Release Date'].value_counts().sort_values(axis=0).head(20)

1915    1
1920    1
1931    1
1935    1
1958    1
1934    1
1937    1
1930    1
1947    1
1927    1
1941    1
1929    1
1948    2
1944    2
1916    2
1925    2
1949    2
1955    2
1943    2
1950    2
Name: Release Date, dtype: int64

Eliminate Films before 1990:
- Looking at our year values, we determined that many of the films before 1990 would act as outliers in our analysis. Ultimately, we determined that our stakeholders are seeking analysis on more current trends in the film indusry, so we eliminated the films from before 1990 from our dataset.

In [11]:
# Removing films released before 1990
MovieBudget_Prepared = MovieBudget_Prepared[(MovieBudget_Prepared['Release Date']>1990)].reset_index(drop=True)
MovieBudget_Prepared

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892
3,4,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,4.243841
4,5,2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,4.153696
...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000
5176,79,1999,Following,6000,48482,240495,234495,40.082500
5177,80,2005,Return to the Land of Wonders,5000,1338,1338,-3662,0.267600
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000


##### Explore Risk Bins with IQR Analysis:
Finally, for this section, we wanted to get a sense for the spread of the 'Production Budget' distribution. We ran a .describe() method on the dataframe and found the following values, which we will use later in our analysis.

In [12]:
MovieBudget_Prepared['Production Budget'].describe()

count    5.180000e+03
mean     3.394279e+07
std      4.336466e+07
min      1.100000e+03
25%      5.575000e+06
50%      2.000000e+07
75%      4.500000e+07
max      4.250000e+08
Name: Production Budget, dtype: float64

Now our dataset from The Numbers is cleaned and prepared to merge with our IMDb dataset after we analyzed the category options for movie types.

#### 2. Exploring Categorization Options for Films and Merging our Datasets.
Within the IMDb dataset, we identified the 'genres' column contained in the 'movie_basics' table and the names of all the directors associated with each film in the 'directors column, which we will integrate later. 

We first connected to the 'movie_basics' table and applied our data cleaning functions to standardize the 'genres' column like we did before for The Numbers dataset. Then we reconfigured the dataset as a pandas DataFrame, allowing us to more easily manipulate and merge the data with our exisiting dataset. Column headers were changed in order to match our exisiting dataset's headers, allowing us to join our datasets on this shared header appropriately.

In [13]:
# Connecting to Movie Basics Table where genres reside.
movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""
movie_basics_df = pd.read_sql(movie_basics_query, conn)

# Applying functions to clean the column headers and to cast the 'genres' column as a list separated by commas.
clean_column_headers(movie_basics_df)
movie_basics_df['Genres'] = clean_comma_series(movie_basics_df['Genres'])
movie_basics_df

# Configuring the SQL table as a pandas DataFrame.
movie_genres_df = pd.DataFrame([movie_basics_df['Movie Id'], movie_basics_df['Primary Title'], movie_basics_df['Genres']])
movie_genres_df = movie_genres_df.transpose()

# Replacing the column header 'Primary Title' with 'Movie' to match our existing dataset column containing the same info.
movie_genres_df.columns = movie_genres_df.columns.str.replace('Primary Title', 'Movie')
movie_genres_df = movie_genres_df.drop_duplicates('Movie')

# Displaying the newly created dataframe containing the 'Movie' title and its corresponding 'Genre(s)'
movie_genres_df

,Movie Id,Movie,Genres
0,tt0063540,Sunghursh,"[Action, Crime, Drama]"
1,tt0066787,One Day Before the Rainy Season,"[Biography, Drama]"
2,tt0069049,The Other Side of the Wind,[Drama]
3,tt0069204,Sabse Bada Sukh,"[Comedy, Drama]"
4,tt0100275,The Wandering Soap Opera,"[Comedy, Drama, Fantasy]"
...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,[Drama]
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,[Documentary]
146141,tt9916706,Dankyavar Danka,[Comedy]
146142,tt9916730,6 Gunn,None


The IMDb data contains many more movies than we have financial information for in The Numbers dataset. We determined that we would only analyze films with sufficient finicial data.

When we merged the datasets, we were left with ONLY the movies that had financial data that we could analyze. 

In [14]:
# Merging The Numbers dataset with the IMDb dataset on the column 'Movie'
Merged_df = MovieBudget_Prepared.merge(movie_genres_df, on = 'Movie', how='left')
Merged_df

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,[Horror]
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,"[Action, Adventure, Fantasy]"
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,"[Action, Adventure, Sci-Fi]"
3,4,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,4.243841,tt2395427,"[Action, Adventure, Sci-Fi]"
4,5,2017,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,4.153696,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,"[Horror, Sci-Fi, Thriller]"
5176,79,1999,Following,6000,48482,240495,234495,40.082500,NaN,NaN
5177,80,2005,Return to the Land of Wonders,5000,1338,1338,-3662,0.267600,NaN,NaN
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,"[Drama, Horror, Thriller]"


Because the 'Genres' column cateroizes some of the films in multiple genres and also contains null values, further data cleaning was required.

- First we exploded the 'Genres' column, separating each unique genre for each film and creating new records to show each film in each of its respective genres.


- We determined that a film falling into multiple genres needs to be counted for each respective genre that it belongs to, as the film can be duplicated along the genre categorization, without duplicating any of the financial metrics. This is because we do not combine genres via addition or any other equation throughout the rest of our analysis. 


- Then we dropped null values, as we are not able to analyze the films that are missing this key data point.

In [15]:
# Further explanation of dropped values.
MergedExploded_df1 = Merged_df.explode('Genres')
MergedExploded_df1

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,Horror
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Action
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Adventure
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Fantasy
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,Action
...,...,...,...,...,...,...,...,...,...,...
5177,80,2005,Return to the Land of Wonders,5000,1338,1338,-3662,0.267600,NaN,NaN
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Drama
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Horror
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Thriller


In [16]:
# Explode Genres column to split any movies that are in multiple Genres.
MergedExploded_df = Merged_df.explode('Genres')

# Drop any movies that have a null value in the Genres column, because we cannot run our
#     analysis without a Genre designation.
MergedExploded_df.dropna(subset=['Genres'], inplace=True)

# Displaying our results to check if we were successful.
MergedExploded_df

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,Horror
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Action
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Adventure
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Fantasy
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,Action
...,...,...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Sci-Fi
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Thriller
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Drama
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Horror


Now, with this cleaned and prepared dataset, we are able to move forward and strategize how to best present this data to our stakeholders.

#### 3. Strategizing how to Present our Findings to the Stakeholders.

We explored which film genre on average produces the highest return on investment by various methods that grouped the genre categories and then calulated the mean ARR values for each genre.

We determined that the range of budgets within the 'Production Budget' column was too wide to compare these film genres accurately, so we decided to create 4 distinct bins defined by the quartiles of the 'Production Budget' column. These bins represent 4 investment ranges that can be considered more risky as the amount of funds invested increases. 

Our analysis determined our 4 risk level production budget investment ranges to be based off the quartiles of the 'Production Budget' column:
- Level 1: 0 - 5,575,000
- Level 2: 5,575,000 - 20,000,000
- Level 3: 20,000,000 - 45,000,000
- Level 4: 45,000,000+

In [17]:
# First we sorted the ARR to see which individual films had the highest ROIS.
MovieBudget_NoBin = MergedExploded_df.copy()
MovieBudget_NoBin.sort_values('ARR', ascending=False, inplace=True)
#MovieBudget_NoBin.reset_index(drop=True, inplace=True)
MovieBudget_NoBin

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
5084,80,2015,The Gallows,100000,22764410,41656474,41556474,416.564740,tt2309260,Horror
5084,80,2015,The Gallows,100000,22764410,41656474,41556474,416.564740,tt2309260,Mystery
5084,80,2015,The Gallows,100000,22764410,41656474,41556474,416.564740,tt2309260,Thriller
4676,12,2012,The Devil Inside,1000000,53262945,101759490,100759490,101.759490,tt1560985,Horror
4897,60,2009,Home,500000,15433,44793168,44293168,89.586336,tt2075392,Drama
...,...,...,...,...,...,...,...,...,...,...
4036,93,2015,Let's Kill Ward's Wife,5000000,0,0,-5000000,0.000000,tt2980708,Crime
4039,96,2012,Should've Been Romeo,5000000,0,0,-5000000,0.000000,tt1717210,Comedy
4039,96,2012,Should've Been Romeo,5000000,0,0,-5000000,0.000000,tt1717210,Drama
4040,97,2015,Aztec Warrior,5000000,0,0,-5000000,0.000000,tt2237914,Action


We ran some exploratory analysis to see how many films are in each genre.

In [18]:
MergedExploded_df['Genres'].value_counts()

Drama          1104
Comedy          646
Action          538
Adventure       413
Thriller        378
Crime           301
Horror          267
Romance         258
Documentary     176
Biography       173
Sci-Fi          172
Mystery         165
Fantasy         148
Family          128
Animation       119
Music            65
History          62
Sport            49
War              30
Western          16
Musical          14
News              1
Name: Genres, dtype: int64

Next, we analyzed each genre's average return ratio based on the initial investment, sorted in a descending manner.

In [19]:
MergedExploded_df.groupby(by='Genres').mean()['ARR'].sort_values(ascending=False)

Genres
Mystery        8.121784
Horror         7.046584
Thriller       4.838324
News           3.894215
Romance        3.524639
Animation      3.502847
Music          3.475146
Biography      3.116844
Comedy         3.039161
Sci-Fi         3.023010
Adventure      2.994670
Fantasy        2.976817
Drama          2.850792
Sport          2.733471
Family         2.655699
Action         2.580998
Documentary    2.401363
Crime          1.929617
History        1.905893
Musical        1.781934
War            1.730887
Western        1.469226
Name: ARR, dtype: float64

The average ARR is pulled down significantly by the many movies that did not make any revenue and it is also affected by certain film genres that had one or two movies total contributing to the average return ratio calculation, thus not adding data points that are support by an adequate number of films.

In [20]:
MergedExploded_df.groupby(by='Genres').mean()['Production Budget'].sort_values(ascending=False)

Genres
Adventure      9.572879e+07
Animation      8.843109e+07
Fantasy        7.666757e+07
Sci-Fi         7.602439e+07
Action         7.168318e+07
Family         5.497129e+07
Western        4.446875e+07
Comedy         3.533861e+07
Musical        3.385000e+07
Crime          2.888937e+07
History        2.853952e+07
Thriller       2.781811e+07
War            2.679333e+07
Drama          2.432458e+07
Biography      2.390655e+07
Mystery        2.296433e+07
Sport          2.292194e+07
Romance        2.092452e+07
Documentary    1.947396e+07
Horror         1.778017e+07
Music          1.617338e+07
News           1.200000e+07
Name: Production Budget, dtype: float64

Because these ARR calculation account for the entire range of production budgets, we felt that presenting this data alone would not be sufficient or fully accurate for our stakeholders. So, we decided to group or film genres by ranges of production budgets or initial investments made/risk taken to make these films by the production companies.

Example 1: the 'Horror' genre has the second highest ARR, and the third lowest average production budget overall. This would suggest that this genre is relatively less risky than almost all of the other genres to produce and it appears to result in almost the highest return on that investment. 

Example 2: Adventure movies are the most expensive to make and their return on investment is ranked 11th in the overall analysis. This means, that Adventure movies require a large initial investment, which a new studio may not choose to risk for its early ventures. 

#### Determing Risk Level Ranges

Recall the spread we found for the initial 'MovieBudget_Prepared' 'Production Budget' column.

This spread includes as many datapoints as we had available to us in The Numbers dataset, therefore offering the most accurate spread of the industry production budget data that we could access. Yes, we ultimately lost some of the film titles as we further cleaned the data and merged this data with the IMDb dataset, but to calculate the spread of the 'Production Budget' column after the 'Genres' category was exploded would create duplicate values and ultimately misrepresent our data.

We have provided the spread below from our original calculation.

In [21]:
# Recall the spread we found for the initial 'MovieBudget_Prepared' 'Production Budget' column.
MovieBudget_Prepared['Production Budget'].describe()

count    5.180000e+03
mean     3.394279e+07
std      4.336466e+07
min      1.100000e+03
25%      5.575000e+06
50%      2.000000e+07
75%      4.500000e+07
max      4.250000e+08
Name: Production Budget, dtype: float64

#### Binning the datasets based off our investment risk levels.

##### Level 1

In [22]:
L1_lower = 0
L1_upper = 5575000

Level_1 = MergedExploded_df[(MergedExploded_df['Production Budget'] <= L1_upper) & (MergedExploded_df['Production Budget'] > L1_lower)]
Level_1

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
3886,17,1996,Shine,5500000,35811509,36672493,31172493,6.667726,tt4064912,Drama
3886,17,1996,Shine,5500000,35811509,36672493,31172493,6.667726,tt4064912,Music
3887,18,2013,Don Jon,5500000,24477704,41268579,35768579,7.503378,tt2229499,Comedy
3887,18,2013,Don Jon,5500000,24477704,41268579,35768579,7.503378,tt2229499,Drama
3887,18,2013,Don Jon,5500000,24477704,41268579,35768579,7.503378,tt2229499,Romance
...,...,...,...,...,...,...,...,...,...,...
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Sci-Fi
5175,78,2018,Red 11,7000,0,0,-7000,0.000000,tt7837402,Thriller
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Drama
5178,81,2015,A Plague So Pleasant,1400,0,0,-1400,0.000000,tt2107644,Horror


After our initial splitting binning of the data, we ran another value counts on the films at this risk level to get a sense for the distribution of films at this risk level.

In [23]:
# Getting the count of movies in each genre at Level 1.
Level_1['Genres'].value_counts()

Drama          321
Comedy         155
Horror         132
Thriller       121
Romance         78
Documentary     77
Action          65
Mystery         61
Crime           58
Sci-Fi          43
Adventure       34
Biography       23
Family          22
Fantasy         22
Music           22
Sport           14
History         11
Musical          5
War              4
Western          4
Animation        3
Name: Genres, dtype: int64

In [24]:
# Sorting the ARR calculations for Level 1 across all genres present
Level_1.groupby(by='Genres').mean()['ARR'].sort_values(ascending=False)

Genres
Mystery        16.075685
Horror         10.256561
Thriller        8.648374
Biography       4.766268
Sport           4.704873
Fantasy         4.578752
Romance         4.508545
Drama           3.248659
Music           3.090963
Comedy          2.965519
Sci-Fi          2.195603
Adventure       2.165493
Documentary     1.847462
Action          1.738472
Crime           1.470922
Family          1.456968
Animation       1.418518
Western         0.972421
History         0.945955
War             0.494678
Musical         0.487783
Name: ARR, dtype: float64

#### Setting the median as a parameter for genres to continue to be relavant to our analysis.
After this inital exploration, we determined that genres with fewer films were appearing higher in the ARR calculation, though their ARR may not be accurately weighted due to a smaller count. 

We decided to set a parameter to eliminate any film genres that had a count equal to or less than the median at each level, this effectively eliminated 11 film genres per risk level and left only the top ten film genres with the most data points contributing to their ARR.

For example: the Biography genre has the 4th highest ARR with only 23 films as compared to the Romance genre which has the 7th highest ARR but 78 films. The difference in ARR's between these two genres is ~.26, however with Romane producing 55 more films, it would appear that Romance is less risky as the data supports the fact that it is more likely that producing a Romance film will result in this relatively high ARR.

Below, we've demonstrated how this has changed our final values to reflect genres with a high count (greater than the median) of films and a high ARR.

In [25]:
# This is to tell us what the median is, the 50% figure
Level_1['Genres'].value_counts().describe()

count     21.000000
mean      60.714286
std       74.300836
min        3.000000
25%       14.000000
50%       34.000000
75%       77.000000
max      321.000000
Name: Genres, dtype: float64

In [26]:
# Elminating anything at or below the median
Level_1_Median = list(Level_1['Genres'].value_counts()[:10].index)
Level_1_Filtered = Level_1.loc[Level_1['Genres'].isin(Level_1_Median)]
Level_1_Filtered['Genres'].value_counts()

Drama          321
Comedy         155
Horror         132
Thriller       121
Romance         78
Documentary     77
Action          65
Mystery         61
Crime           58
Sci-Fi          43
Name: Genres, dtype: int64

In [27]:
# Final calculation of sorted ARR values by genre.
Level_1_Filtered.groupby(by='Genres').mean()['ARR'].sort_values(ascending=False)

Genres
Mystery        16.075685
Horror         10.256561
Thriller        8.648374
Romance         4.508545
Drama           3.248659
Comedy          2.965519
Sci-Fi          2.195603
Documentary     1.847462
Action          1.738472
Crime           1.470922
Name: ARR, dtype: float64

#### Running a linear regression to further support our ARR calculations.

Next we wanted to further support our analysis resulting in a top three films at this category of Mystery, Horror, and Thriller, resepctively. 

We calculated a linear regression and have displayed the resulting coefficients below. The coefficient values do not match the ARR values above, but we did not expect them to match perfectly. Rather, we were interested in the relative order of the coefficients, as a higher coefficient represents a higher weighting of that genre, and thus a likelihood that that genre will consistently produce a higher ARR.

As you can see below: Mystery, Horror, and Thriller have the highest coefficient values. 

In [28]:
# Linear Regression coefficients 
# Every coefficient associated with a genre is a weight that indicates its significance toward ARR
# The order of the coefficients per genre is consistent with the order of the ARR means
results1 = smf.ols(formula="ARR ~ C(Genres)", data=Level_1_Filtered).fit()
results1.summary()
print(results1.params.sort_values(ascending=False))

C(Genres)[T.Mystery]        14.337213
C(Genres)[T.Horror]          8.518089
C(Genres)[T.Thriller]        6.909902
C(Genres)[T.Romance]         2.770073
Intercept                    1.738472
C(Genres)[T.Drama]           1.510187
C(Genres)[T.Comedy]          1.227047
C(Genres)[T.Sci-Fi]          0.457131
C(Genres)[T.Documentary]     0.108990
C(Genres)[T.Crime]          -0.267550
dtype: float64


We looped through each of the 3 remaining risk levels next. We performed all of the same calculations for each of the risk levels, so to save space we have not copied our Markdown again for each risk level, as the logic remains the same. Comments have been included in the code cells for a brief reminder.

##### Level 2

In [29]:
L2_lower = 5575000
L2_upper = 20000000

Level_2 = MergedExploded_df[(MergedExploded_df['Production Budget'] < L2_upper) & (MergedExploded_df['Production Budget'] > L2_lower)]
Level_2

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
2602,17,2018,Bird Box,19800000,0,0,-19800000,0.000000,tt2737304,Drama
2602,17,2018,Bird Box,19800000,0,0,-19800000,0.000000,tt2737304,Horror
2602,17,2018,Bird Box,19800000,0,0,-19800000,0.000000,tt2737304,Sci-Fi
2604,19,2017,The Shape of Water,19500000,63859435,195790794,176290794,10.040554,tt5580390,Adventure
2604,19,2017,The Shape of Water,19500000,63859435,195790794,176290794,10.040554,tt5580390,Drama
...,...,...,...,...,...,...,...,...,...,...
3874,3,2012,Trade of Innocents,5800000,15091,15091,-5784909,0.002602,tt1772408,Drama
3874,3,2012,Trade of Innocents,5800000,15091,15091,-5784909,0.002602,tt1772408,Thriller
3882,11,2010,Standing Ovation,5600000,531806,531806,-5068194,0.094965,tt1303803,Musical
3883,12,2012,Khiladi 786,5600000,385422,14683763,9083763,2.622101,tt2166214,Action


In [30]:
# This is to tell us what the median is, the 50% figure
Level_2['Genres'].value_counts().describe()

count     22.000000
mean      52.818182
std       70.054153
min        1.000000
25%       15.000000
50%       32.000000
75%       66.500000
max      324.000000
Name: Genres, dtype: float64

In [31]:
# Elminating anything at or below the median
Level_2_Median = list(Level_2['Genres'].value_counts()[:10].index)
Level_2_Filtered = Level_2.loc[Level_2['Genres'].isin(Level_2_Median)]
Level_2_Filtered['Genres'].value_counts()

Drama        324
Comedy       142
Thriller      90
Crime         79
Romance       74
Horror        67
Biography     65
Action        63
Mystery       38
Adventure     36
Name: Genres, dtype: int64

In [32]:
# Final calculation of sorted ARR values by genre.
Level_2_Filtered.groupby(by='Genres').mean()['ARR'].sort_values(ascending=False)

Genres
Horror       5.106763
Mystery      4.896437
Thriller     3.791583
Romance      3.256027
Biography    3.253414
Comedy       3.068696
Action       3.034745
Drama        2.931449
Adventure    2.409342
Crime        1.799999
Name: ARR, dtype: float64

In [33]:
# Linear Regression coefficients 
# Every coefficient associated with a genre is a weight that indicates its significance toward ARR
# The order of the coefficients per genre is consistent with the order of the ARR means
results2 = smf.ols(formula="ARR ~ C(Genres)", data=Level_2_Filtered).fit()
results2.summary()
print(results2.params.sort_values(ascending=False))

Intercept                 3.034745
C(Genres)[T.Horror]       2.072018
C(Genres)[T.Mystery]      1.861692
C(Genres)[T.Thriller]     0.756837
C(Genres)[T.Romance]      0.221281
C(Genres)[T.Biography]    0.218668
C(Genres)[T.Comedy]       0.033951
C(Genres)[T.Drama]       -0.103297
C(Genres)[T.Adventure]   -0.625403
C(Genres)[T.Crime]       -1.234746
dtype: float64


<Image>

##### Level 3

In [34]:
L3_lower = 20000000
L3_upper = 45000000

Level_3 = MergedExploded_df[(MergedExploded_df['Production Budget'] < L3_upper) & (MergedExploded_df['Production Budget'] >= L3_lower)]
Level_3

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
1309,27,2012,Argo,44500000,136025503,227140757,182640757,5.104287,tt1024648,Biography
1309,27,2012,Argo,44500000,136025503,227140757,182640757,5.104287,tt1024648,Drama
1309,27,2012,Argo,44500000,136025503,227140757,182640757,5.104287,tt1024648,Thriller
1312,31,2010,Charlie St. Cloud,44000000,31206263,48478084,4478084,1.101775,tt1438254,Drama
1312,31,2010,Charlie St. Cloud,44000000,31206263,48478084,4478084,1.101775,tt1438254,Fantasy
...,...,...,...,...,...,...,...,...,...,...
2597,12,2016,The History of Love,20000000,0,476624,-19523376,0.023831,tt0443533,Romance
2597,12,2016,The History of Love,20000000,0,476624,-19523376,0.023831,tt0443533,War
2599,14,2014,Dwegons and Leprechauns,20000000,0,0,-20000000,0.000000,tt1134666,Animation
2600,15,2016,Fight Valley,20000000,0,0,-20000000,0.000000,tt4280822,Action


In [35]:
# This is to tell us what the median is, the 50% figure
Level_3['Genres'].value_counts().describe()

count     21.000000
mean      60.428571
std       69.047499
min        1.000000
25%       20.000000
50%       36.000000
75%       79.000000
max      290.000000
Name: Genres, dtype: float64

In [36]:
# Elminating anything at or below the median
Level_3_Median = list(Level_3['Genres'].value_counts()[:10].index)
Level_3_Filtered = Level_3.loc[Level_3['Genres'].isin(Level_3_Median)]
Level_3_Filtered['Genres'].value_counts()

Drama          290
Comedy         174
Action         133
Crime          103
Thriller        87
Romance         79
Adventure       63
Biography       60
Horror          42
Documentary     39
Name: Genres, dtype: int64

In [37]:
# Final calculation of sorted ARR values by genre.
Level_3_Filtered.groupby(by='Genres').mean()['ARR'].sort_values(ascending=False)

Genres
Horror         3.043820
Romance        3.001602
Documentary    2.939782
Comedy         2.849206
Drama          2.533341
Thriller       2.508820
Biography      2.317956
Adventure      2.250723
Action         2.172090
Crime          2.122229
Name: ARR, dtype: float64

In [38]:
# Linear Regression coefficients 
# Every coefficient associated with a genre is a weight that indicates its significance toward ARR
# The order of the coefficients per genre is consistent with the order of the ARR means
results3 = smf.ols(formula="ARR ~ C(Genres)", data=Level_3_Filtered).fit()
results3.summary()
print(results3.params.sort_values(ascending=False))

Intercept                   2.172090
C(Genres)[T.Horror]         0.871730
C(Genres)[T.Romance]        0.829512
C(Genres)[T.Documentary]    0.767692
C(Genres)[T.Comedy]         0.677116
C(Genres)[T.Drama]          0.361251
C(Genres)[T.Thriller]       0.336730
C(Genres)[T.Biography]      0.145866
C(Genres)[T.Adventure]      0.078633
C(Genres)[T.Crime]         -0.049861
dtype: float64


##### Level 4

In [39]:
L4_lower = 45000000
L4_upper = 1000000000

Level_4 = MergedExploded_df[(MergedExploded_df['Production Budget'] < L4_upper) & (MergedExploded_df['Production Budget'] >= L4_lower)]
Level_4

,Id,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross,Profitability,ARR,Movie Id,Genres
0,1,2009,Avatar,425000000,760507625,2776345279,2351345279,6.532577,tt1775309,Horror
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Action
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Adventure
1,2,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,2.546673,tt1298650,Fantasy
2,3,2019,Dark Phoenix,350000000,42762350,149762350,-200237650,0.427892,tt6565702,Action
...,...,...,...,...,...,...,...,...,...,...
1306,24,2002,Pinocchio,45000000,3681811,31681811,-13318189,0.704040,tt1488589,Fantasy
1306,24,2002,Pinocchio,45000000,3681811,31681811,-13318189,0.704040,tt1488589,Musical
1308,26,2012,Foodfight!,45000000,0,73706,-44926294,0.001638,tt0249516,Action
1308,26,2012,Foodfight!,45000000,0,73706,-44926294,0.001638,tt0249516,Animation


In [40]:
# This is to tell us what the median is, the 50% figure
Level_4['Genres'].value_counts().describe()

count     21.000000
mean      72.238095
std       84.402550
min        4.000000
25%       12.000000
50%       30.000000
75%       82.000000
max      280.000000
Name: Genres, dtype: float64

In [41]:
# Elminating anything at or below the median
Level_4_Median = list(Level_4['Genres'].value_counts()[:10].index)
Level_4_Filtered = Level_4.loc[Level_4['Genres'].isin(Level_4_Median)]
Level_4_Filtered['Genres'].value_counts()

Adventure    280
Action       277
Comedy       175
Drama        169
Animation     90
Fantasy       82
Thriller      80
Sci-Fi        79
Crime         61
Family        52
Name: Genres, dtype: int64

In [42]:
# Final calculation of sorted ARR values by genre.
Level_4_Filtered.groupby(by='Genres').mean()['ARR'].sort_values(ascending=False)

Genres
Animation    3.881066
Sci-Fi       3.578737
Adventure    3.338000
Comedy       3.269292
Action       2.871839
Thriller     2.786542
Family       2.586350
Fantasy      2.560044
Drama        2.485187
Crime        2.208390
Name: ARR, dtype: float64

In [43]:
# Linear Regression coefficients 
# Every coefficient associated with a genre is a weight that indicates its significance toward ARR
# The order of the coefficients per genre is consistent with the order of the ARR means
results4 = smf.ols(formula="ARR ~ C(Genres)", data=Level_4_Filtered).fit()
results4.summary()
print(results4.params.sort_values(ascending=False))

Intercept                 2.871839
C(Genres)[T.Animation]    1.009227
C(Genres)[T.Sci-Fi]       0.706897
C(Genres)[T.Adventure]    0.466161
C(Genres)[T.Comedy]       0.397453
C(Genres)[T.Thriller]    -0.085297
C(Genres)[T.Family]      -0.285489
C(Genres)[T.Fantasy]     -0.311795
C(Genres)[T.Drama]       -0.386652
C(Genres)[T.Crime]       -0.663449
dtype: float64


## Top Directors for Risk Level 1:
### Remaining Data Munging for Director Analysis is: `notebooks/claire_dev/tableau_data_prep.ipynb`

In [44]:
# Add Production Budget Bucket Level Column to all of our data that was split out by level:
Level_1_Filtered['prod_budget_bucket'] = 'b1'
Level_2_Filtered['prod_budget_bucket'] = 'b2'
Level_3_Filtered['prod_budget_bucket'] = 'b3'
Level_4_Filtered['prod_budget_bucket'] = 'b4'

# Merge all levels together:
movie_risk_genre = pd.concat([Level_1_Filtered, Level_2_Filtered, Level_3_Filtered, Level_4_Filtered])
# Remove the Exploded-out Genre column by first only selecting the cols we need and then de-duping the rows:
movie_risk_NO_genre = movie_risk_genre[['Movie Id', 'Movie', 'Release Date', 'ARR', 'prod_budget_bucket']] 
movie_risk_NO_genre.drop_duplicates()

,Movie Id,Movie,Release Date,ARR,prod_budget_bucket
3886,tt4064912,Shine,1996,6.667726,b1
3887,tt2229499,Don Jon,2013,7.503378,b1
3888,tt1999890,Hell Fest,2018,3.294406,b1
3891,tt1606618,Iris,2001,1.014633,b1
3896,tt1508675,Le Havre,2011,2.450945,b1
...,...,...,...,...,...
1298,tt0376136,The Rum Diary,2011,0.478772,b4
1300,tt3289724,Welcome to Marwen,2018,0.286109,b4
1301,tt1634122,Johnny English Reborn,2011,3.658676,b4
1306,tt1488589,Pinocchio,2002,0.704040,b4


- When we drilled down into Risk Level One, looking at each of the top 10 films by ARR, we found that some of the top directors are all associated with the same studio that mae The Gallows, Paranormal Activity and insidious. 7/10 of these top 10 films are [horror/mystery/thriller] or horror.


![Top Directors for Investment Level 1](images/top_directors_L1.png)

## Conclusion & Recommendations

- Produce Horror or Mystery movies at any budget level.
- Top directors: Chris Lofing, William B. Bell, Jordan Peele 
- At higher risk levels, consider comedy and romance.
- At highest risk level, best performing genres are animation, sci-fi and adventure
